<h1>Projet Python - Facteurs d'influence de l'immobilier </h1>
<hr/>

<h2>Contexte</h2>
<p>
Tom a choisit le sujet donc il doit travailler plus. Nous traitons des logements et étudions pleins de facteurs divers et variés.
</p>

<h2>I - Récupération et analyse des données</h2>
<p>
On récupère les données utiles et téléchargeons les module sutiles dans notre projets par la suite. ON nettoie aussi les données. C'est une partie très importantes.
</p>


<h5>1) Modules utilisés</h5>
Plusieurs modules
</p>




In [1]:
import os
import s3fs
import pandas as pd


<h5>2) Téléchargement des données</h5>
<p>
On récupère données
</p>

In [2]:
#importation des données dvf géolocalisées en csv

S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]

fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={"endpoint_url": S3_ENDPOINT_URL}
)

with fs.open("renan/diffusion/dvf.csv", mode="rb") as f:
    df = pd.read_csv(f)

/tmp/ipykernel_1839/2755059622.py:11: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [ ]:
#On récupère désormais les données de population ; cependant certaines valeurs ne sont pas renseignés. On remarque aussi que certaines communes ont des populations trop faibles
#pour que le ratio considéré ventes entre 2020 et 2050/population en 2019 ait du sens ; en particulier, la commune de Bezonvaux a zéro habitants !!! 
# On impose donc un nombre d'habitants minimum arbitraire, et on exclut les communes de populations trop faibles ou non renseignés

df_pop=pd.read_excel("popcommunes.xlsx")
print(df_pop)

cols = ["p19_pop"]
df_pop[cols].isna().sum()
lignes_na = df_pop[df_pop[cols].isna().any(axis=1)]
print(lignes_na)
print(lignes_na.shape)


<h5>3) Nettoyage des données</h5>
<p>
Nettoyons atsiquons un peu ces données
</p>

In [3]:
#On se limite aux maisons et appartements

df = df[df['type_local'].isin(['Maison', 'Appartement'])]
print(df)
df=df.reset_index()

          id_mutation date_mutation  numero_disposition nature_mutation  \
17             2020-6    2020-07-02                   1           Vente   
31             2020-8    2020-07-01                   1           Vente   
32             2020-9    2020-07-02                   1           Vente   
36            2020-11    2020-07-03                   1           Vente   
37            2020-12    2020-07-02                   1           Vente   
...               ...           ...                 ...             ...   
20102723  2025-502875    2025-06-23                   1           Vente   
20102725  2025-502876    2025-06-25                   1           Vente   
20102726  2025-502877    2025-06-25                   1           Vente   
20102734  2025-502880    2025-06-27                   1           Vente   
20102735  2025-502881    2025-06-27                   1           Vente   

          valeur_fonciere  adresse_numero adresse_suffixe  \
17                77000.0           34

In [4]:
comptage_df = df['id_mutation'].value_counts().reset_index()
comptage_df.columns = ['id_mutation', 'nombre_de_lignes']
print(comptage_df)

id_uniques = comptage_df[comptage_df['nombre_de_lignes'] == 1]
print(id_uniques.shape)

print(id_uniques)
df_sans_lots=df[df['id_mutation'].isin(id_uniques['id_mutation'])]
print(df_sans_lots)

         id_mutation  nombre_de_lignes
0        2021-687176              9290
1        2022-919567              7657
2        2022-909252              7477
3        2021-342324              7112
4        2020-491811              6173
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4861637 rows x 2 columns]
(4133749, 2)
         id_mutation  nombre_de_lignes
727888   2023-581082                 1
727889   2023-510332                 1
727890   2023-580244                 1
727891   2023-510334                 1
727892   2023-579686                 1
...              ...               ...
4861632    2022-7412                 1
4861633    2022-7410                 1
4861634    2022-7409                 1
4861635    2022-7405                 1
4861636    2022-7419                 1

[4133749 rows x 2 colu

In [5]:
#les codes communes n'ont pas tous le même type ; certains sont des ints, d'autres des strings. On convertit l'ensemble en string pour plus de cohérence(on 
#ne peut pas faire l'opération inverse ; les codes corses commencent par 2A ou 2B !)
print(inttostr('433'))

df.loc[:,'code_commune'] = df['code_commune'].apply(inttostr)
print(df['code_commune'])

NameError: name 'inttostr' is not defined

In [ ]:
#Calcul du nombre d'annonces avec des identifiants distincts pour chaque commune

ventes_par_commune = df['code_commune'].value_counts().reset_index(name='nombre')

#Résultats : on remarque on particulier que Toulouse est la commune comptant le plus de ventes entre 2020 et 2025(50684) !!!

print(ventes_par_commune)

print( ventes_par_commune["nombre"][ventes_par_commune["code_commune"]=='2A004'])

print(ventes_par_commune['nombre'].max())

print(ventes_par_commune['nombre'].idxmax())

print(ventes_par_commune['nombre'][ventes_par_commune['nombre'].idxmax()])


In [ ]:
#On joint les données en se restreignant aux communes dont on connaît les populations et de populations pas trop faibles. Le min est arbitraire est peut être changé
# On utilise les données de la population
df_pop.rename(columns={'codgeo': 'code_commune'}, inplace=True)

ventes_par_commune_par_habitant=pd.merge(ventes_par_commune, df_pop, on='code_commune')
ventes_par_commune_par_habitant = ventes_par_commune_par_habitant[
    (ventes_par_commune_par_habitant['p19_pop'].notna()) &
    (ventes_par_commune_par_habitant['p19_pop'] > 1000)
]
ventes_par_commune_par_habitant['ventes par habitants par commune']=ventes_par_commune_par_habitant['nombre']/ventes_par_commune_par_habitant['p19_pop']

print(ventes_par_commune_par_habitant['ventes par habitants par commune'])

print(ventes_par_commune_par_habitant['ventes par habitants par commune'].max())

print(ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax())

print(ventes_par_commune_par_habitant['code_commune'][ventes_par_commune_par_habitant['ventes par habitants par commune'].idxmax()])

In [ ]:
#tentative de récupération des prix au m2
#Remarque : on ne considère que le ratio valeur foncière/surface batîe de la maison ou de l'appartement, puisqu'on ignore les dépendances ou autres terrains additionnels.
#on sur-estime donc la valeur au m2. On ignore les lots(on utilise df_sans_lots qui ne contient pas les observations dont l'id_mutation apparait plusieurs fois)
#à la fois parce qu'on ne peut pas retrouver la valeur foncière des éléments individuels, et pour se concentrer sur les logements anciens
df_sans_lots = df_sans_lots.copy()
print(df_sans_lots.shape)

df_sans_lots = df_sans_lots[
    (df_sans_lots['surface_reelle_bati'].notna()) &
    (df_sans_lots['valeur_fonciere'].notna()) &
    (df_sans_lots['surface_reelle_bati'] > 10)
]



df_sans_lots['rapport valeur foncière et surface bâtie']=df_sans_lots['valeur_fonciere']/df_sans_lots['surface_reelle_bati']

print(df_sans_lots['rapport valeur foncière et surface bâtie'])

print(df_sans_lots['rapport valeur foncière et surface bâtie'].max())

print(df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax())

print(df_sans_lots.loc[df_sans_lots['rapport valeur foncière et surface bâtie'].idxmax()])

#problème ; valeurs monstrueuses de certains logements, visiblement des immeubles entiers plutôt que des logements(ci-dessus,immeuble en construction de presque 1 milliard...)


moyenne_par_commune=df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

print(moyenne_par_commune['rapport valeur foncière et surface bâtie'].max())

print(moyenne_par_commune['code_commune'].loc[moyenne_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])



# Calcul de la médiane par commune
mediane_par_commune = df_sans_lots.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Valeur maximale de la médiane
print(mediane_par_commune['rapport valeur foncière et surface bâtie'].max())

# Code de la commune avec la médiane maximale
print(mediane_par_commune['code_commune'][mediane_par_commune['rapport valeur foncière et surface bâtie'].idxmax()])

In [ ]:
#meilleure alternative au troncage précédent ; on tronque par commune, on touve alors des médianes et moyennes très proches, maximale pour le sixième arrondissement parisien, c'est cohérent !

# Calculer les quantiles 0.025 et 0.975 par commune
quantiles_par_commune = (
    df_sans_lots
    .groupby('code_commune')['rapport valeur foncière et surface bâtie']
    .quantile([0.025, 0.975])
    .unstack()
    .reset_index()
    .rename(columns={0.025: 'quantile_025', 0.975: 'quantile_975'})
)

print(quantiles_par_commune)

# Fusionner les quantiles avec df_sans_lots
df_sans_lots_tronqué = df_sans_lots.merge(
    quantiles_par_commune,
    on='code_commune',
    how='left'
)

df_sans_lots_tronqué = df_sans_lots_tronqué[
    (df_sans_lots_tronqué['rapport valeur foncière et surface bâtie'] >= df_sans_lots_tronqué['quantile_025']) &
    (df_sans_lots_tronqué['rapport valeur foncière et surface bâtie'] <= df_sans_lots_tronqué['quantile_975'])
]

# Calcul de la moyenne par commune (tronqué)
moyenne_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].mean().reset_index()

# Affichage de la moyenne maximale et de la commune associée
print(moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(moyenne_par_commune_tronqué['code_commune'].loc[moyenne_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])

# Calcul de la médiane par commune (tronqué)
mediane_par_commune_tronqué = df_sans_lots_tronqué.groupby('code_commune')['rapport valeur foncière et surface bâtie'].median().reset_index()

# Affichage de la médiane maximale et de la commune associée
print(mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].max())
print(mediane_par_commune_tronqué['code_commune'][mediane_par_commune_tronqué['rapport valeur foncière et surface bâtie'].idxmax()])